In [72]:
import os
import pandas as pd

filepath = "enron/enron1/"
text_data = {"subject": [], "content": [], "spam": []}
for file in os.listdir(filepath + "spam"):
    with open(filepath + "spam/" + file, "r", encoding='UTF-8', errors='ignore') as f:
        lines = f.readlines()
        text_data['subject'].append(lines[0][9:].replace("\n", ""))
        text_data['content'].append("".join(lines[1:]))
        text_data["spam"].append(1)

for file in os.listdir(filepath + "ham"):
    with open(filepath + "ham/" + file, "r", encoding='UTF-8') as f:
        lines = f.readlines()
        text_data['subject'].append(lines[0][9:].replace("\n", ""))
        text_data['content'].append("".join(lines[1:]))
        text_data["spam"].append(0)

text_data = pd.DataFrame(text_data)
print(sum(text_data["spam"]))
text_data

1500


,subject,content,spam
0,dobmeos with hgh my energy level has gone up !...,introducing\ndoctor - formulated\nhgh\nhuman g...,1
1,your prescription is ready . . oxwq s f e,"low cost prescription medications\nsoma , ultr...",1
2,get that new car 8434,people nowthe weather or climate in any partic...,1
3,await your response,"dear partner ,\nwe are a team of government of...",1
4,"coca cola , mbna america , nascar partner with...",stock\nprofile\nabout\ncompany\ninvestment\nhi...,1
...,...,...,...
5167,re : tenaska iv,i ' ll call you on thursday . . . what ' s a g...,0
5168,generic contract,"hi daren ,\nsorry for the delay . here is a bl...",0
5169,re : contracts and credit,thanks - - i ' ll include it in the master fil...,0
5170,re : tenaska iv,"ok , since we don ' t appear to be able to hoo...",0


In [84]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
import numpy as np

X_train_raw, X_test_raw, Y_train, Y_test = train_test_split(text_data[["subject", 'content']], text_data[["spam"]],
                                                            test_size=0.2,
                                                            random_state=2022)

tv1 = TfidfVectorizer(max_features=5000)
tv2 = CountVectorizer(max_features=5000)
X_train_subject, X_train_content = tv1.fit_transform(X_train_raw["subject"]).toarray(), tv2.fit_transform(
    X_train_raw["content"]).toarray()
X_test_subject, X_test_content = tv1.transform(X_test_raw["subject"]).toarray(), tv2.transform(
    X_test_raw["content"]).toarray()
X_train = np.concatenate((X_train_subject, X_train_content), axis=1)
X_test = np.concatenate((X_test_subject, X_test_content), axis=1)

In [91]:
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import GridSearchCV

parameters = {"n_estimators": [50, 100, 200]}
clf = RandomForestClassifier()
grid = GridSearchCV(clf, parameters, scoring="accuracy")
grid.fit(X_train, Y_train)
grid.best_params_

D:\Python310\lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
D:\Python310\lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
D:\Python310\lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
D:\Python310\lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please ch

{'n_estimators': 200}

In [92]:
from sklearn.metrics import classification_report

clf = RandomForestClassifier(n_estimators=200)
clf.fit(X_train, Y_train)
Y_pred = clf.predict(X_test)
print(classification_report(Y_pred, Y_test))

C:\Users\99480\AppData\Local\Temp\ipykernel_24196\859583954.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf.fit(X_train, Y_train)


              precision    recall  f1-score   support

           0       0.98      0.98      0.98       735
           1       0.96      0.96      0.96       300

    accuracy                           0.98      1035
   macro avg       0.97      0.97      0.97      1035
weighted avg       0.98      0.98      0.98      1035



In [93]:
X_train.shape

(4137, 9452)